## Investigating the Zillow Housing Price Dataset

N. Ranjan & R. Mattson | CSCI 6430* | Mar 11, 2021

Total Runtime = 2mins

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2score
from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import mean_absolute_percentage_error as mape
#from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Lasso,Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder

###  Retrieve Data and Prepare Dataset

In [2]:
#TODO - get from google docs?? - We should!
DF1B = pd.read_csv(r'Datasets/City_MedianRentalPrice_1Bedroom.csv')
DF2B = pd.read_csv(r'Datasets/City_MedianRentalPrice_2Bedroom.csv')
DF3B = pd.read_csv(r'Datasets/City_MedianRentalPrice_3Bedroom.csv')
DF4B = pd.read_csv(r'Datasets/City_MedianRentalPrice_4Bedroom.csv')

In [3]:
# remove columns with useless information - These were only indexes
DF1B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF2B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF3B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF4B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)

In [4]:
# record the number of bedrooms within the frame
DF1B.insert(0, 'BedroomsSq', 1)
DF2B.insert(0, 'BedroomsSq', 2)
DF3B.insert(0, 'BedroomsSq', 3)
DF4B.insert(0, 'BedroomsSq', 4)
#BE CAREFUL WITH INSERTING BEDROOMS
insertBedrooms = np.where(DF1B.columns=='2010-02')[0][0] 
print(int(insertBedrooms))
DF1B.insert(int(insertBedrooms), 'Bedrooms', 1)
DF2B.insert(int(insertBedrooms), 'Bedrooms', 2)
DF3B.insert(int(insertBedrooms), 'Bedrooms', 3)
DF4B.insert(int(insertBedrooms), 'Bedrooms', 4)

5


In [5]:
frames = [DF1B,DF2B,DF3B,DF4B]
result = pd.concat(frames)
result = result.sample(frac=1) #Shuffle!
result = result.reset_index(drop=True) #Reset Index

In [6]:
result

,BedroomsSq,RegionName,State,Metro,CountyName,Bedrooms,2010-02,2010-03,2010-04,2010-05,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,1,Mooresville,NC,Charlotte-Concord-Gastonia,Iredell County,1,NaN,NaN,NaN,NaN,...,990.0,1020.0,1021.5,1040.0,1051.0,1039.5,1032.5,1027.5,1030.0,1030.0
1,2,Ferndale,MI,Detroit-Warren-Dearborn,Oakland County,2,NaN,NaN,NaN,NaN,...,1150.0,1149.0,1200.0,1200.0,1300.0,1350.0,1350.0,1350.0,1272.5,1325.0
2,1,San Leandro,CA,San Francisco-Oakland-Hayward,Alameda County,1,NaN,NaN,NaN,NaN,...,1897.5,1950.0,1926.5,2075.0,1995.0,2125.0,1995.0,1924.0,1925.5,1895.0
3,2,Farmers Branch,TX,Dallas-Fort Worth-Arlington,Dallas County,2,NaN,NaN,NaN,NaN,...,1946.0,1947.0,1830.0,1735.0,1815.0,1815.0,1804.0,1859.5,1935.0,1938.5
4,1,Malibu,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,1,NaN,NaN,NaN,NaN,...,4900.0,5000.0,5000.0,5000.0,5000.0,4900.0,5000.0,5000.0,5000.0,5000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,3,Missouri City,TX,Houston-The Woodlands-Sugar Land,Fort Bend County,3,NaN,NaN,NaN,NaN,...,1568.0,1595.0,1590.0,1600.0,1620.0,1600.0,1600.0,1600.0,1580.0,1599.5
3385,1,Morgantown,WV,Morgantown,Monongalia County,1,NaN,NaN,NaN,NaN,...,600.0,630.0,600.0,682.5,682.5,650.0,650.0,650.0,650.0,650.0
3386,3,Grandview,MO,Kansas City,Jackson County,3,NaN,NaN,NaN,NaN,...,1065.0,1045.0,1068.0,1012.5,1000.0,1065.0,1107.5,1157.5,1097.5,1147.5
3387,2,Fremont,CA,San Francisco-Oakland-Hayward,Alameda County,2,NaN,NaN,NaN,NaN,...,2725.0,2770.0,2800.0,2796.5,2750.0,2728.0,2700.0,2666.5,2651.0,2734.5


In [7]:
# result.to_csv(r'Datasets\City_MedianRentalPrice_AllHomes.csv')
result.to_csv(r'Datasets/City_MedianRentalPrice_AllHomes_ALTERED.csv')

In [8]:
del DF1B,DF2B,DF3B,DF4B
del result,frames
#Just saving memory, let's continue with our final dataset

### Preproccess Data

In [9]:
# DF = pd.read_csv(r'Datasets\City_MedianRentalPrice_AllHomes.csv')
DF = pd.read_csv(r'Datasets/City_MedianRentalPrice_AllHomes_ALTERED.csv',index_col=0)
# TrainDF = DF.iloc[:,:-1].copy() # Rest of the columns come here as training data
# TestDF = DF.iloc[:,-1].copy()   # We predicting the last column
# del DF


In [10]:
ST = DF['State'].value_counts()
ST

CA    564
FL    426
TX    248
GA    167
VA    146
MA    135
WA    124
IL    103
MD    102
NJ     98
CO     82
CT     79
NC     79
OH     76
MI     75
NY     74
PA     70
MN     52
SC     52
MO     49
IN     46
OR     43
UT     37
AZ     35
KS     35
IA     32
OK     29
LA     27
TN     26
WI     23
RI     21
NV     20
HI     20
AL     18
KY     17
ID     17
NM     16
MS     15
ND     14
AR     14
DE     13
MT     13
NE     12
WV     10
AK      8
NH      6
SD      6
WY      4
ME      4
DC      4
VT      3
Name: State, dtype: int64

In [11]:
lb=LabelEncoder()
lb.fit(DF['State'])

LabelEncoder()

In [12]:

RecordAnalysisDF = pd.DataFrame(columns=['RunNumber','Title','Algorithm','R2Mean','R2Median','RMSEMean','RMSEMedian']) #Will save results for later. You can keep running code below it and each time validation is run, results will be added here.
runNo = 1

In [13]:
isBedStateOptimized = True #Leave this as True.
cut_off = 0.25 # if number of nulls > 25%, just remove the column.
#We got lucky here as the null values always increased the further we go in the past 

In [14]:
row_count = DF.index.size
NullCountDF = pd.DataFrame({'Null_Count':DF.isnull().sum().to_numpy(),'Col_Name':DF.columns.to_numpy()})
colsToKeep = NullCountDF[NullCountDF.Null_Count<=row_count*cut_off].Col_Name.to_numpy() #columns to keep,rest have too many nulls
DF = DF[colsToKeep].copy() 
print(DF.isnull().sum().to_numpy())
print(colsToKeep)

[  0   0   0   0   0   0 829 800 771 741 695 621 571 510 465 412 361 306
 249 209 144 107  60   0   0   0   0   0   0   0   0   0   0   0   0]
['BedroomsSq' 'RegionName' 'State' 'Metro' 'CountyName' 'Bedrooms'
 '2017-08' '2017-09' '2017-10' '2017-11' '2017-12' '2018-01' '2018-02'
 '2018-03' '2018-04' '2018-05' '2018-06' '2018-07' '2018-08' '2018-09'
 '2018-10' '2018-11' '2018-12' '2019-01' '2019-02' '2019-03' '2019-04'
 '2019-05' '2019-06' '2019-07' '2019-08' '2019-09' '2019-10' '2019-11'
 '2019-12']


In [15]:
#Removing Nulls
def RemoveNulls(TheDF):
   print('Removing Nulls...')
   for col in TheDF.columns:
      for x in TheDF[TheDF[col].isnull()].index: #Going through all columns
         beds = TheDF.loc[x]['Bedrooms']
         state = TheDF.loc[x]['State']
         #print)
         if(isBedStateOptimized == True):
            valFill = TheDF[(TheDF['Bedrooms'] == beds) & (TheDF['State']==state)][col] #Bedroom and state
            if np.all(np.isnan(valFill)):
               valFill = TheDF[(TheDF['Bedrooms'] == beds)][col] #Check if we can do only with bedroom as this is more correlated than state
            if np.all(np.isnan(valFill)):
               valFill = TheDF[(TheDF['State'] == beds)][col] #If only bedrooms didnt work, try with State
            if np.all(np.isnan(valFill)):
               valFill =TheDF[col] #If we get null for those too..just take median of whole column
            valFill = np.nanmedian(valFill)
            TheDF.loc[x,col] = valFill
         else:
            valFill =TheDF[col]
            valFill = np.nanmedian(valFill)
            TheDF.loc[x,col] = valFill
   print('Nulls Removed.')

In [16]:
RemoveNulls(DF)

Removing Nulls...
Nulls Removed.


In [17]:
DF

,BedroomsSq,RegionName,State,Metro,CountyName,Bedrooms,2017-08,2017-09,2017-10,2017-11,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,1,Mooresville,NC,Charlotte-Concord-Gastonia,Iredell County,1,911.50,865.0,884.0,875.0,...,990.0,1020.0,1021.5,1040.0,1051.0,1039.5,1032.5,1027.5,1030.0,1030.0
1,2,Ferndale,MI,Detroit-Warren-Dearborn,Oakland County,2,1250.00,1270.0,1266.0,1237.0,...,1150.0,1149.0,1200.0,1200.0,1300.0,1350.0,1350.0,1350.0,1272.5,1325.0
2,1,San Leandro,CA,San Francisco-Oakland-Hayward,Alameda County,1,1700.00,1695.0,1722.5,1775.0,...,1897.5,1950.0,1926.5,2075.0,1995.0,2125.0,1995.0,1924.0,1925.5,1895.0
3,2,Farmers Branch,TX,Dallas-Fort Worth-Arlington,Dallas County,2,1759.00,1672.0,1605.5,1641.0,...,1946.0,1947.0,1830.0,1735.0,1815.0,1815.0,1804.0,1859.5,1935.0,1938.5
4,1,Malibu,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,1,1795.00,1769.0,1794.0,1775.0,...,4900.0,5000.0,5000.0,5000.0,5000.0,4900.0,5000.0,5000.0,5000.0,5000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,3,Missouri City,TX,Houston-The Woodlands-Sugar Land,Fort Bend County,3,1550.00,1500.0,1500.0,1597.5,...,1568.0,1595.0,1590.0,1600.0,1620.0,1600.0,1600.0,1600.0,1580.0,1599.5
3385,1,Morgantown,WV,Morgantown,Monongalia County,1,586.25,670.0,650.0,595.0,...,600.0,630.0,600.0,682.5,682.5,650.0,650.0,650.0,650.0,650.0
3386,3,Grandview,MO,Kansas City,Jackson County,3,1050.00,1050.0,1050.0,1050.0,...,1065.0,1045.0,1068.0,1012.5,1000.0,1065.0,1107.5,1157.5,1097.5,1147.5
3387,2,Fremont,CA,San Francisco-Oakland-Hayward,Alameda County,2,2610.50,2600.0,2600.0,2600.0,...,2725.0,2770.0,2800.0,2796.5,2750.0,2728.0,2700.0,2666.5,2651.0,2734.5


In [18]:
#Assumed Last Column is Target, it should return Xtrain,Ytrain,Xtest,Ytest
def ChangeDataSetForTimeSeries(TheDF):
    """
    Uses a sliding window mechanism to convert a dataset into a timeseries model.
    With T=10, we look at past 10 months data to predict the 11th month throughout the available     history of the dataset.
    """
    posOfFirstDate = np.where(TheDF.columns=='Bedrooms')[0][0] + 1
    FirstDate = TheDF.columns[posOfFirstDate]
    T = 10
    Startindex = np.where(TheDF.columns==FirstDate)[0][0]
    #print(TheDF.columns[Startindex])
    TrainEndindex = TheDF.columns.size-3
    #ValidationEndindex = TheDF.columns.size-3
    TestEndindex = TheDF.columns.size-2
    #print(TheDF.columns[TrainEndindex])
    size = TrainEndindex - Startindex + 1
    colListTrain = TheDF.columns[0:Startindex].tolist()
    for i in range(T):
        colListTrain.append('T'+str(i))
    TrainDFX = pd.DataFrame(columns=colListTrain[:-1])
    TrainDFY = pd.DataFrame(columns=['T'+str(T)])
    #print(colListTrain,len(colListTrain))
    #Make Train-------------------------------
    X_Arr = []
    Y_Arr = []
    for i in range(len(TheDF.index)): #Row Iteration
        if(i%25==0):
            print('\r', 'Iteration', i+1, ' / Rows:', len(TheDF.index), end='')
        initdataToInsert = TheDF.iloc[i,0:Startindex].to_numpy().tolist() #All our categorical data
        #print('-')
        #print(initdataToInsert)
        #print('-')
        for t in range(Startindex,TrainEndindex + 1 - T):
            x = TheDF.iloc[i,t:t+T].to_numpy().tolist()
            #print(x)
            #x = np.concatenate((initdataToInsert,x))
            X_Arr.extend(initdataToInsert)
            X_Arr.extend(x)
            y = TheDF.iloc[i,t+T]
            Y_Arr.append(y)
            #return X_Arr,Y_Arr
        #--------Inefficient Garbage----------------
        # X_Arr = np.array(X_Arr).reshape(-1,len(colListTrain))
        # Y_Arr = np.array(Y_Arr).reshape(-1,1)
        
        # X_Arr = pd.DataFrame(X_Arr,columns=colListTrain).to_dict()
        # Y_Arr = pd.DataFrame(Y_Arr,columns=['T10'])
        # #print(X_Arr)
        # TrainDFX = TrainDFX.append(X_Arr,ignore_index=True) #TOO SLOW IN A LOOP
        # TrainDFY = TrainDFY.append(Y_Arr,ignore_index=True)
        #break
        #return TrainDFX,TrainDFY
        #for ind in ():
            #print(X)
            #FinalDFX.loc[ind] = X[:,ind]
            #FinalDFY.loc[ind] = Y.reshape(-1,1)
            #break
        #break   
        # --------------------------    
    print('\r', 'Iteration', i+1, ' / Rows:', len(TheDF.index), end='') #Handy loading msg
    X_Arr = np.array(X_Arr).reshape(-1,len(colListTrain)) #We had appended data in a single dimension array, time to get it back to a table form. (Unflatten)
    Y_Arr = np.array(Y_Arr).reshape(-1,1)
    
    X_Arr = pd.DataFrame(X_Arr,columns=colListTrain)
    Y_Arr = pd.DataFrame(Y_Arr,columns=['T10'])
    #print(X_Arr)
    TrainDFX = TrainDFX.append(X_Arr,ignore_index=True)
    TrainDFY = TrainDFY.append(Y_Arr,ignore_index=True)
    #Training Sets Done!!!!!
    testColNames = TheDF.columns[0:Startindex].to_numpy().tolist()
    #valColData = TheDF.columns[0:Startindex].to_numpy().tolist()
    testColData = TheDF.columns[0:Startindex].to_numpy().tolist()
    for i in range(T):
        testColNames.append('T'+str(i))
        #valColData.extend([TheDF.columns[ValidationEndindex-T+i+1]])
        testColData.extend([TheDF.columns[TestEndindex-T+i+1]])
    #Get Data for Testing.
    print('----')
    #print(valColData,testColData)
    #ValDFX = TheDF[valColData].copy()
    #ValDFX.columns = valAndTestColNames
    #ValDFY = TheDF.iloc[:,ValidationEndindex+1].copy()
    #ValDFY.columns = 'T'+str(T)
    # ValidationEndindex
    TestDFX = TheDF[testColData].copy()
    TestDFX.columns = testColNames
    TestDFY = TheDF.iloc[:,TestEndindex+1].copy()
    TestDFY.columns = 'T'+str(T)
    #return TrainDFX,TrainDFY,ValDFX,ValDFY,TestDFX,TestDFY
    return TrainDFX,TrainDFY,TestDFX,TestDFY
    pass

In [19]:
TrainDFX,TrainDFY,TestDFX,TestDFY = ChangeDataSetForTimeSeries(DF) #Finally, got train and testing data

 Iteration 3389  / Rows: 3389----


In [20]:
TrainDFX

,BedroomsSq,RegionName,State,Metro,CountyName,Bedrooms,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9
0,1,Mooresville,NC,Charlotte-Concord-Gastonia,Iredell County,1,911.5,865.0,884.0,875.0,875.0,867.0,875.0,880.0,877.5,913.0
1,1,Mooresville,NC,Charlotte-Concord-Gastonia,Iredell County,1,865.0,884.0,875.0,875.0,867.0,875.0,880.0,877.5,913.0,922.5
2,1,Mooresville,NC,Charlotte-Concord-Gastonia,Iredell County,1,884.0,875.0,875.0,867.0,875.0,880.0,877.5,913.0,922.5,967.0
3,1,Mooresville,NC,Charlotte-Concord-Gastonia,Iredell County,1,875.0,875.0,867.0,875.0,880.0,877.5,913.0,922.5,967.0,935.0
4,1,Mooresville,NC,Charlotte-Concord-Gastonia,Iredell County,1,875.0,867.0,875.0,880.0,877.5,913.0,922.5,967.0,935.0,955.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57608,4,Little Elm,TX,Dallas-Fort Worth-Arlington,Denton County,4,1995.0,1947.5,1895.0,1900.0,1990.0,2000.0,2100.0,1995.0,1997.5,2100.0
57609,4,Little Elm,TX,Dallas-Fort Worth-Arlington,Denton County,4,1947.5,1895.0,1900.0,1990.0,2000.0,2100.0,1995.0,1997.5,2100.0,2099.5
57610,4,Little Elm,TX,Dallas-Fort Worth-Arlington,Denton County,4,1895.0,1900.0,1990.0,2000.0,2100.0,1995.0,1997.5,2100.0,2099.5,2000.0
57611,4,Little Elm,TX,Dallas-Fort Worth-Arlington,Denton County,4,1900.0,1990.0,2000.0,2100.0,1995.0,1997.5,2100.0,2099.5,2000.0,1897.5


In [21]:
del DF #Save memory, also makes sure that i dont accidentally use this dataframe

In [22]:
#lb = LabelEncoder()

In [23]:
#Found out Bedroom was more correlated than these and then on 2nd place is State
# TrainDF['RegionName'] = lb.fit_transform(TrainDF['RegionName'])
# TrainDF['State'] = lb.fit_transform(TrainDF['State'])
# TrainDF['Metro'] = lb.fit_transform(TrainDF['Metro'])
# TrainDF['CountyName'] = lb.fit_transform(TrainDF['CountyName'])
#TrainDF['State'] = lb.fit_transform(TrainDF['State'])

In [24]:
#TrainDF.corr()

In [25]:
#Pass by Value Reference Magic!
def AddStateSparseMatrix(TrainingDF):
   """
   We are taking the State column and replacing it with 51 columns, (50 US states + DC).
   """
   enc = OneHotEncoder(handle_unknown='ignore',sparse = False)
   SM = enc.fit_transform(TrainingDF['State'].to_numpy().reshape(-1,1)) #Got it!
   for ind,cat in zip(range(0,len(enc.categories_[0])),enc.categories_[0]):
      TrainingDF.insert(0,'is'+str(cat),SM[:,ind]) # Inserted column names are isAK, isAL, isDC..etc. And it will contain 0 if that is not the state and 1 if that is the state
   TrainingDF.drop(columns=['State'],inplace=True)

### Run Models

In [26]:
def RunAllModels(TrainDF,Output,title):
    """
    Trains and validates data on multiple models
    """
    global runNo
    global RecordAnalysisDF
    global isBedStateOptimized
    def PrintMetrics(scores,algo):
        """
        Internal Function
        Used to print on console, nothing fancy here.
        """
        global RecordAnalysisDF
        print('-'*5)
        meanR2 = np.absolute(scores['test_r2']).mean()
        medianR2 = np.median(np.absolute(scores['test_r2']))
        meanRMSE = np.absolute(scores['test_neg_root_mean_squared_error']).mean()
        medianRMSE = np.median(np.absolute(scores['test_neg_root_mean_squared_error']))
        meanMAPE = np.absolute(scores['test_neg_mean_absolute_percentage_error']).mean()
        medianMAPE = np.median(np.absolute(scores['test_neg_mean_absolute_percentage_error']))
        print('Average R2 score ', meanR2)
        print('Median R2 score ', medianR2)
        print('Average Root Mean Square Error ', meanRMSE)
        print('Median Root Mean Square Error ', medianRMSE)
        print('-'*5)
        if(isBedStateOptimized==False):
            algo = algo + ' Nulls removed with complete column'
        RecordAnalysisDF = RecordAnalysisDF.append({'RunNumber':runNo,'Title':title,'Algorithm':algo,'R2Mean':meanR2,'R2Median':medianR2,'RMSEMean':meanRMSE,'RMSEMedian':medianRMSE},ignore_index=True)
        #print('Average Mean Absolute Percentage Error ', meanMAPE)
        #print('Median Mean Absolute Percentage Error ', medianMAPE)
    #-----------------------Linear Regression---------------------------------------------
    print(title)
    print('-'*10)
    print('Linear Regression')
    model = make_pipeline(StandardScaler(),LinearRegression())
    # Y_pred = model.predict(X_test)
    # Y_pred = scY.inverse_transform(Y_pred)
    # Y_test = scY.inverse_transform(Y_test)
    scores = cross_validate(model, TrainDF, Output, cv=15,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
    PrintMetrics(scores,'Linear Regression')#,meanMAPE,medianMAPE)
    #----------------------------Polynomial Lasso amd Ridge-------------------------
    for _,degree in enumerate([1]): #Couldn't do enumerate([1,2]) please do it if you have 10-15mins to spare and a powerful system. Tried it a few times, got bad scores so we decided to not do it.
        polymodel = make_pipeline(StandardScaler(),PolynomialFeatures(degree),Lasso(alpha=1e-2,max_iter=1e+6,tol=1e-2))
        scores = cross_validate(polymodel, TrainDF, Output, cv=15,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
        print('Polynomial Degree with Lasso: '+str(degree))
        PrintMetrics(scores,'Lasso Degree ' + str(degree))

        polymodel = make_pipeline(StandardScaler(),PolynomialFeatures(degree),Ridge(alpha=1e-3,max_iter=1e+6))
        scores = cross_validate(polymodel, TrainDF, Output, cv=15,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
        print('Polynomial Degree with Ridge: '+str(degree))
        PrintMetrics(scores,'Ridge Degree ' + str(degree))
    #----------------------------DecisionTreeRegressor---------------------------------------
    model = make_pipeline(StandardScaler(),DecisionTreeRegressor(max_depth=100))
    scores = cross_validate(model, TrainDF, Output, cv=15,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
    print('DecisionTreeRegressor')
    PrintMetrics(scores,'DecisionTreeRegressor')
    runNo = runNo + 1
    print('-'*5)
    print('-'*15)

### Method 1: Drop all the Categorical columns

In [27]:
FinalTrainDF = TrainDFX.drop(columns=['RegionName', 'Metro', 'CountyName'])
# RemoveNulls(FinalTrainDF,True)
FinalTrainDF.drop(columns=['State'],inplace=True)
RunAllModels(FinalTrainDF,TrainDFY,'Dropped Categorical Data')

Dropped Categorical Data
----------
Linear Regression
-----
Average R2 score  0.9744413955835195
Median R2 score  0.9908098078832884
Average Root Mean Square Error  157.3611320259808
Median Root Mean Square Error  125.92456710445619
-----
Polynomial Degree with Lasso: 1
-----
Average R2 score  0.9620602686560296
Median R2 score  0.9872678447400793
Average Root Mean Square Error  180.9302802979508
Median Root Mean Square Error  138.2863592994292
-----
Polynomial Degree with Ridge: 1
-----
Average R2 score  0.9744415702023288
Median R2 score  0.9908099221224903
Average Root Mean Square Error  157.36103438427259
Median Root Mean Square Error  125.9261625593911
-----
DecisionTreeRegressor
-----
Average R2 score  0.9483884070891156
Median R2 score  0.9793625126070374
Average Root Mean Square Error  255.17971039953824
Median Root Mean Square Error  202.78298355354622
-----
-----
---------------


### Method2: Next, Let's change States to a sparse matrix and then to 51 columns, 1 for each State. Lets drop the other categorical columns

In [28]:
FinalTrainDF = TrainDFX.drop(columns=['RegionName', 'Metro', 'CountyName'])
#RemoveNulls(FinalTrainDF,True)
AddStateSparseMatrix(FinalTrainDF)
#FinalTrainDF.drop(columns=['State'],inplace=True)
RunAllModels(FinalTrainDF,TrainDFY,'One Hot Encoded State,Removed Others')
#FinalTrainDF

One Hot Encoded State,Removed Others
----------
Linear Regression
-----
Average R2 score  0.9744690111648431
Median R2 score  0.9908048678324737
Average Root Mean Square Error  157.32342214402595
Median Root Mean Square Error  125.77333918637196
-----
Polynomial Degree with Lasso: 1
-----
Average R2 score  0.9621803812668402
Median R2 score  0.9872781293151955
Average Root Mean Square Error  180.8637223739935
Median Root Mean Square Error  138.3461114326286
-----
Polynomial Degree with Ridge: 1
-----
Average R2 score  0.9744705270421042
Median R2 score  0.9908070657918399
Average Root Mean Square Error  157.3242197367044
Median Root Mean Square Error  125.80284401256488
-----
DecisionTreeRegressor
-----
Average R2 score  0.9493479105193878
Median R2 score  0.9790232787563821
Average Root Mean Square Error  251.10436043858454
Median Root Mean Square Error  188.1090834119632
-----
-----
---------------


In [29]:
RecordAnalysisDF

,RunNumber,Title,Algorithm,R2Mean,R2Median,RMSEMean,RMSEMedian
0,1,Dropped Categorical Data,Linear Regression,0.974441,0.990810,157.361132,125.924567
1,1,Dropped Categorical Data,Lasso Degree 1,0.962060,0.987268,180.930280,138.286359
2,1,Dropped Categorical Data,Ridge Degree 1,0.974442,0.990810,157.361034,125.926163
3,1,Dropped Categorical Data,DecisionTreeRegressor,0.948388,0.979363,255.179710,202.782984
4,2,"One Hot Encoded State,Removed Others",Linear Regression,0.974469,0.990805,157.323422,125.773339
5,2,"One Hot Encoded State,Removed Others",Lasso Degree 1,0.962180,0.987278,180.863722,138.346111
6,2,"One Hot Encoded State,Removed Others",Ridge Degree 1,0.974471,0.990807,157.324220,125.802844
7,2,"One Hot Encoded State,Removed Others",DecisionTreeRegressor,0.949348,0.979023,251.104360,188.109083


In [30]:
RecordAnalysisDF.index +=1

In [31]:
RecordAnalysisDF.to_csv('RecordAnalysis.csv',index_label='S.No.') # Save to csv for analysis

In [32]:
#After Analyzing this CSV File (Copy of the table is in the report), we found that "One Hot Encoded State,Removed Others - NAN removed with column with Linear Regression" had the least R2Mean and RMSE Median. So, it had performed best in the 2/4 metrics we considered. Hence, we decided to go ahead with it.

### Testing

In [33]:
#One Hot Encoded State,Removed Others	Linear Regression
FinalTrainDF = TrainDFX.drop(columns=['RegionName', 'Metro', 'CountyName'])
#RemoveNulls(FinalTrainDF,True)
AddStateSparseMatrix(FinalTrainDF)
FinalTestDF = TestDFX.drop(columns=['RegionName', 'Metro', 'CountyName'])
#FinalTrainDF[ValDF.name] = ValDF
#RemoveNulls(FinalTrainDF,False)
AddStateSparseMatrix(FinalTestDF)
#FinalTrainDF.drop(columns=['State'],inplace=True)

In [34]:
model = LinearRegression()
hist = model.fit(FinalTrainDF,TrainDFY)
YPred = model.predict(FinalTestDF)

In [35]:
R2Score = r2score(TestDFY,YPred)
RMSE = mse(TestDFY,YPred,squared=False)
print('R2 score ', R2Score)
print('Root Mean Square Error ', RMSE)
print('Average of Output ',TestDFY.mean())

R2 score  0.9944658263673406
Root Mean Square Error  136.35530883611855
Average of Output  1779.1479787547948


In [37]:
for bed in range(1,5):
    filterArr = (FinalTestDF['Bedrooms'] == bed)
    #print(filterArr)
    print('For '+ str(bed) +' bedrooms')
    print('-'*3)
    R2Score = r2score(TestDFY[filterArr],YPred[filterArr])
    RMSE = mse(TestDFY[filterArr],YPred[filterArr],squared=False)
    print('R2 score ', R2Score)
    print('Root Mean Square Error ', RMSE)
    print('Average of '+ str(bed) +' Bedroom Cost ',TestDFY[filterArr].mean())
    print('Number of Houses ',len(TestDFY[filterArr].index))
    print('-'*10)

For 1 bedrooms
---
R2 score  0.9913823922628504
Root Mean Square Error  49.1272876332863
Average of 1 Bedroom Cost  1384.4834963325184
Number of Houses  818
----------
For 2 bedrooms
---
R2 score  0.9922283282885305
Root Mean Square Error  59.51255172112318
Average of 2 Bedroom Cost  1566.38814229249
Number of Houses  1265
----------
For 3 bedrooms
---
R2 score  0.9963037370743136
Root Mean Square Error  96.81088377653744
Average of 3 Bedroom Cost  1969.8116161616163
Number of Houses  990
----------
For 4 bedrooms
---
R2 score  0.9935542811880613
Root Mean Square Error  386.7985016699112
Average of 4 Bedroom Cost  3055.159810126582
Number of Houses  316
----------
